## 多工具并行使用

In [25]:
from langchain.tools import tool
import json
import requests


@tool
def add(first_int: int, second_int: int) -> int:
    "将两个整数相加。"
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "对底数求指数幂。"
    return base ** exponent


@tool
def multiply(first_int: int, second_int: int) -> int:
    """将两个整数相乘。"""
    return first_int * second_int


@tool
def divide(first_int: int, second_int: int) -> float:
    """将两个整数相除。"""
    return first_int / second_int


@tool
def wannianli(date: str) -> dict:
    "根据指定日期查询其农历，习俗，星期几，假期，生肖"
    date = date.replace("-0", "-")
    print(date)
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://v.juhe.cn/calendar/day"
    params = {
        "key": "43d6c167a771d2d32a139283c9cf9648",  # 在个人中心->我的数据,接口名称上方查看
        "date": date,  # 指定日期,格式为YYYY-MM-DD,如月份和日期小于10,则取个位,如:2012-1-1
    }
    resp = requests.get(url, params, headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json


@tool
def tianqi(city: str) -> dict:
    """查询最近几日的天气情况，包括温度，天气，湿度，风向等"""
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://apis.juhe.cn/simpleWeather/query"
    params = {
        "key": "395bac783edef3f02663bbb491a060a6",  # 在个人中心->我的数据,接口名称上方查看
        "city": city,  # 要查询的城市名称或城市ID
    }
    resp = requests.get(url, params, headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json

In [26]:
tools: [tool] = [add, exponentiate, multiply, divide, tianqi, wannianli]

In [30]:
import time
from langchain.tools.render import render_text_description
from langchain.prompts.chat import ChatPromptTemplate

current_date = time.strftime("%Y-%m-%d", time.localtime())
rendered_tools = render_text_description(tools)

system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

今天是{current_date},根据用户输入，返回解决用户问题所需的所有工具的名称和输入，按照执行顺序。
将您的响应作为多个带有'name'和'arguments'键的 JSON blob 返回，“arguments”键对应的值应该是所选函数的输入参数的字典，字典里不要有任何说明。
此JSON blob必须是如下格式：```json...```
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [33]:
from langchain.chat_models.tongyi import ChatTongyi
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatTongyi()
chain_ = {"input": RunnablePassthrough()} | prompt | model | JsonOutputParser()

In [ ]:
chain_.invoke({"input": "3加上1234等于几？明天是农历几号，杭州天气怎样"})

In [34]:
chain_.invoke("3加上5，除以2，乘以8")

OutputParserException: Invalid json output: ```json
[
    {
        "name": "add",
        "arguments": {"first_int": 3, "second_int": 5}
    },
    {
        "name": "divide",
        "arguments": {"first_int": 8, "second_int": 2}
    },
    {
        "name": "multiply",
        "arguments": {"first_int": result_from_divide, "second_int": 8}
    }
]
```
请计算`result_from_divide`后继续执行。

In [16]:
from operator import itemgetter


# 从工具列表中根据模型输出model_output的name属性选取tool对象
# 但是一次只会选取一个，属于单工具调用
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool


chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "3加上1234等于几？"})

1237

### 定义多工具同时调用的方法

In [19]:
from typing import Union
from langchain_core.runnables import (
    Runnable,
    RunnablePassthrough,
    RunnableLambda,
    RunnableMap
)

tool_map = {tool.name: tool for tool in tools}


def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["name"]]
    return RunnablePassthrough.assign(output=itemgetter("arguments") | tool)


call_tool_list = RunnableLambda(call_tool).map()

In [20]:
from langchain_core.runnables import RunnablePassthrough

chain0 = {"input": RunnablePassthrough()} | prompt | model | JsonOutputParser() | call_tool_list


In [21]:
chain0.invoke({"input": "3加上1234等于几？明天是农历几号，杭州天气怎样"})

2024-7-16


[{'name': 'add',
  'arguments': {'first_int': 3, 'second_int': 1234},
  'output': 1237},
 {'name': 'wannianli',
  'arguments': {'date': '2024-07-16'},
  'output': {'reason': 'Success',
   'result': {'data': {'animalsYear': '龙',
     'weekday': '星期二',
     'lunarYear': '甲辰年',
     'lunar': '六月十一',
     'year-month': '2024-7',
     'date': '2024-7-16',
     'suit': '理发.开业.祈福.祭祀.牧养.开光.求子',
     'avoid': '结婚.出行.搬新房.安葬.作灶.入殓.成人礼.上梁',
     'holiday': '',
     'desc': ''}},
   'error_code': 0}},
 {'name': 'tianqi',
  'arguments': {'city': '杭州'},
  'output': {'reason': '查询成功!',
   'result': {'city': '杭州',
    'realtime': {'temperature': '34',
     'humidity': '63',
     'info': '阴',
     'wid': '02',
     'direct': '南风',
     'power': '2级',
     'aqi': '49'},
    'future': [{'date': '2024-07-15',
      'temperature': '26/34℃',
      'weather': '雷阵雨',
      'wid': {'day': '04', 'night': '04'},
      'direct': '西南风'},
     {'date': '2024-07-16',
      'temperature': '27/37℃',
      'weather': '晴

In [22]:
template_nature_language = """根据下面问题和json格式的响应，编写一个针对此问题自然语言回应：
问题：{question}
响应：{response}"""
prompt_response_nl = ChatPromptTemplate.from_template(template_nature_language)
chain1 = {"question": RunnablePassthrough(), "response": chain0} | prompt_response_nl | model | StrOutputParser()


In [23]:
chain1.invoke({"input": "3加上1234等于几？明天是农历几号，杭州天气怎样"})

2024-7-16


'3加1234等于1237。明天是2024年7月16日，农历为六月十一，属龙，星期二，适宜理发、开业、祈福、祭祀等活动，但需避免结婚、出行等。杭州明天的天气预报显示，白天有雷阵雨，气温27到37摄氏度，南风2级；后几天也是晴天或雷阵雨交替，温度在27到38摄氏度左右。'

In [24]:
chain_.invoke({"input": "3加上5，除以2，乘以8"})

{'name': 'add', 'arguments': {'first_int': 3, 'second_int': 5}}